# Homework 5
## Sungwon Lee
### 017473950

## Step 1.

In [1]:
# load Fashion MNIST
import torch
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import FashionMNIST

In [2]:
fashion_mnist = FashionMNIST(root='./homework5/data/', train=True, download=True, \
                             transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))
                             

100.0%


Extracting ./homework5/data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./homework5/data/FashionMNIST\raw



100.0%


Extracting ./homework5/data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./homework5/data/FashionMNIST\raw



100.0%


Extracting ./homework5/data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./homework5/data/FashionMNIST\raw



100.0%

Extracting ./homework5/data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./homework5/data/FashionMNIST\raw



In [3]:
device = torch.device('cpu')

In [ ]:
import matplotlib.pyplot as plt

# Get a sample image and label
img, label = fashion_mnist[0]  # Get the first sample

img = img.to(device)
# Denormalize the image for visualization
def denorm(x):
    return (x * 0.5) + 0.5  # Inverse the normalization

# Convert tensor to numpy array for plotting
img = denorm(img)
img = img.squeeze().numpy()  # Remove extra dimensions and convert to NumPy array

# Plot the image
plt.imshow(img, cmap='gray')
plt.title(f'Label: {label}')
plt.show()

In [3]:
img, label = fashion_mnsit[0]
print('Label: ', label)
print(img[:, 10:15, 10:15])
torch.min(img), torch.max(img)

Label:  9
tensor([[[-1.0000, -1.0000, -1.0000,  0.5137,  0.7882],
         [-0.9765, -1.0000, -0.9059,  0.7176,  0.7255],
         [-0.9529, -1.0000, -0.2235,  0.9137,  0.7412],
         [-1.0000, -1.0000, -0.5686,  0.8510,  0.7882],
         [-1.0000, -1.0000,  0.8588,  0.7725,  0.7020]]])


(tensor(-1.), tensor(1.))

In [4]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
# img_norm = lambda x: ((x+1)/2).clamp(0, 1)
img_norm = denorm(img).squeeze().numpy()
plt.imshow(img_norm, cmap='gray')
